In [1]:
import csv
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import scipy.signal
%matplotlib inline

import nest

# VERSION_LST = [1, 1, 2, 3]
# NREG_LST = [3,5,5,5]
VERSION_LST = [2, 3]
NREG_LST = [5, 5]
GEOM_LST = [
#     [(0, 1), (1, 2)],
#     [(0, 1), (1, 2), (2, 3), (3, 4), (4, 1), (0, 3)],
    [(0, 1), (1, 2), (3, 1), (4, 2)],
    [(0, 1), (1, 2), (2, 0), (0, 3), (3, 2), (3, 4), (4, 1)],
]

WSCALE_LST = [2.0, 4.0, 6.0]
SYND_LST = [50.0, 200.0, 800.0]
# STIM_LST = [(0.0, 0.0), (10.0, 20.0), (10.0, 1.0), (50.0, 20.0), (50.0, 1.0)]
STIM_LST = [(0.0, 0.0)]

gigaPAramLst = []
for i in range(len(VERSION_LST)):
    for W in WSCALE_LST:
        for SYN in SYND_LST:
            for sm, sf in STIM_LST:
                gigaPAramLst += [(VERSION_LST[i], NREG_LST[i], GEOM_LST[i], W, SYN, sm, sf)]
                    
print(len(gigaPAramLst))

18


## Step 1: Simulate mult-layer Brunel Network

In [2]:
for i in range(11, len(gigaPAramLst)):
    VERSION, N_REGION, LL_CONN_REGIONS, WEIGHT_SCALE, SYN_DELAY_LL, STIM_MAG, STIM_FREQ = gigaPAramLst[i]
    print("doing simulation", i)

    # SIMULATION PARAMETERS
    dt = 0.1         # simulation time step size
    tstop = 30000.    # simulation duration 30sec

    ##############################
    # POPULATION PROPERTIES
    ##############################
    N_POPULAITONS = 2 * N_REGION
    NETWORK_SIZE_FACTOR = 1000
    N_E = int(0.8 * NETWORK_SIZE_FACTOR)
    N_I = int(0.2 * NETWORK_SIZE_FACTOR)
    N_NEURON_PP = [N_E, N_I] * N_REGION   # number of neurons in each population

    ##############################
    # Connectivity Properties
    ##############################

    # Connection probabilities
    PCONN_EE = 0.2  # EXC-EXC connectivity probability
    PCONN_EI = 0.2  # EXC-INH connectivity probability
    PCONN_IE = 0.2  # INH-EXC connectivity probability
    PCONN_II = 0.2  # INH-INH connectivity probability
    PCONN_LL = 0.05  # EXC-EXC layer to layer connectivity probability

    # Synapse properties
    W_EXC_MAX = WEIGHT_SCALE
    W_INH_MAX = -4 * WEIGHT_SCALE
    SYN_DELAY_IL = 1.5


    ##############################
    # Stimulus properties
    ##############################
    NOISE_RATE = 6000.
    W_NOISE_MAX = 10.0
    W_NOISE_MAX = [W_NOISE_MAX] * N_POPULAITONS # max input weight of noise


    ##############################
    # Connectivity
    ##############################

    # Inter-layer connectivity graph
    LL_CONN_POPULATIONS = [(2*i, 2*j) for i,j in LL_CONN_REGIONS]
    CONN_GRAPH_POPULATIONS = []

    # Construct connectivity within each layer
    for i in range(N_REGION):
        CONN_GRAPH_POPULATIONS += [
            (2*i,   2*i,   {'WMAX' : W_EXC_MAX, 'PCONN' : PCONN_EE, 'DELAY' : SYN_DELAY_IL}),
            (2*i,   2*i+1, {'WMAX' : W_EXC_MAX, 'PCONN' : PCONN_EI, 'DELAY' : SYN_DELAY_IL}),
            (2*i+1, 2*i,   {'WMAX' : W_INH_MAX, 'PCONN' : PCONN_IE, 'DELAY' : SYN_DELAY_IL}),
            (2*i+1, 2*i+1, {'WMAX' : W_INH_MAX, 'PCONN' : PCONN_II, 'DELAY' : SYN_DELAY_IL})
        ]

    # Construct inter-layer connectivity
    for i,j in LL_CONN_POPULATIONS:    
        CONN_GRAPH_POPULATIONS += [(i, j,   {'WMAX' : W_EXC_MAX, 'PCONN' : PCONN_LL, 'DELAY' : SYN_DELAY_LL})]
    
    ##############################
    # RUN MODEL
    ##############################

    # reset kernel and set dt
    nest.ResetKernel()
    nest.SetKernelStatus({'resolution': dt})

    # create nodes
    neuron_nodes           = [nest.Create('iaf_psc_alpha', n) for n in N_NEURON_PP]
    neuron_spike_detectors = [nest.Create('spike_detector') for i in range(N_POPULAITONS)]
    noise_generator        = nest.Create('poisson_generator', params={'rate' : NOISE_RATE})


    # Connect input to 0th node
    if STIM_MAG > 0.0:
        ac_generator           = nest.Create('ac_generator', params={'amplitude' : STIM_MAG, 'frequency' : STIM_FREQ})
        nest.Connect(ac_generator, neuron_nodes[0], conn_spec='all_to_all')

    # create connections
    for i in range(N_POPULAITONS):
        # Connect detectors to populations
        nest.Connect(neuron_nodes[i], neuron_spike_detectors[i], conn_spec='all_to_all')

        # Connect noise to populations
        nest.Connect(noise_generator, neuron_nodes[i], 
                     conn_spec='all_to_all', 
                     syn_spec={'weight': W_NOISE_MAX[i], 'delay': dt})

    for i, j, param in CONN_GRAPH_POPULATIONS:
        # Connect populations to each other
        nest.Connect(neuron_nodes[i], neuron_nodes[j],
                     conn_spec={'rule': 'pairwise_bernoulli', 'p': param['PCONN']}, 
                     syn_spec={'model':'static_synapse', 'weight': param['WMAX'], 'delay' : param['DELAY']})

    # simulate
    nest.Simulate(tstop)
    
    ##############################
    # PLOT MODEL ANALYSIS
    ##############################
    
    outputName = 'data_V' + str(VERSION)
    outputName += '_N' + str(N_REGION)
    outputName += '_W' + str(WEIGHT_SCALE)
    outputName += '_W' + str(WEIGHT_SCALE)
    outputName += '_DT' + str(SYN_DELAY_LL)
    outputName += '_SM' + str(STIM_MAG)
    outputName += '_SF' + str(STIM_FREQ)

    logFile = open(outputName+"_log.txt", "w")

    fig, ax = plt.subplots(ncols=2, figsize=(10,5))
    for i in range(N_POPULAITONS):
        tspk = nest.GetStatus(neuron_spike_detectors[i])[0]['events']['times']
        nspk = nest.GetStatus(neuron_spike_detectors[i])[0]['events']['senders']
        avgrate = len(tspk) / (tstop / 1000) / N_NEURON_PP[i]
        logFile.write("Ensemble" + str(i) + " avgrate =" + str(int(avgrate*100)/100))

        nBins = 10000
        bins = np.linspace(0, tstop, nBins)
        hist, _ = np.histogram(tspk, bins=bins)
        histNorm = hist * nBins / (tstop / 1000) / N_NEURON_PP[i]
        pxx,f = plt.mlab.psd(histNorm, Fs=1000)  # Power spectral density

        ax[0].plot(bins[:-1], histNorm)
        ax[1].loglog(f, pxx)

    plt.savefig(outputName + '_PSD.png')
    plt.close()
    
    
    ##############################
    # Save traces to file
    ##############################
    with open(outputName +'_spikes.txt', "w") as f:
        writer = csv.writer(f)
        writer.writerow(['populationIdx', 'spiketimes', 'neuronIdx'])

        for i in range(N_POPULAITONS):
            # Only write excitatory population data
            if (i % 2 == 0):
                tspk = nest.GetStatus(neuron_spike_detectors[i])[0]['events']['times']
                nspk = nest.GetStatus(neuron_spike_detectors[i])[0]['events']['senders']

                for t, n in zip(tspk, nspk):
                    writer.writerow([i//2, t, n])


    with open(outputName + '_populations.txt', "w") as f:
        writer = csv.writer(f)
        for i in range(N_POPULAITONS):
            ids = nest.GetStatus(neuron_nodes[i], 'global_id')
            writer.writerow([i, ids[0], ids[-1]])
    
    
    ##############################
    # COMPUTE AND PLOT ENERGY TRANSFER
    ##############################
    #######################################
    # 1. Extract inter-layer connectivity
    #######################################

    # Learn to map from neuron index to population index
    # FIXME: Surely, there is a built-in function
    NEURON_IDXS = [0]
    for i in range(N_POPULAITONS):
        NEURON_IDXS.append(NEURON_IDXS[i] + N_NEURON_PP[i])
    NEURON_IDXS = NEURON_IDXS[1:]
    N_NEURON_TOT = NEURON_IDXS[-1]

    def getPopIdx(neuronIdx):
        i = 0
        while NEURON_IDXS[i] <= neuronIdx:
            i += 1
        return i

    # Find out how many times each neuron maps to another region
    connMap = np.zeros((N_NEURON_TOT, len(LL_CONN_POPULATIONS)))

    for arr in nest.GetConnections():
        src = arr[0]-1
        trg = arr[1]-1

        if (src < N_NEURON_TOT) and (trg < N_NEURON_TOT):
            linkIdx = (getPopIdx(src), getPopIdx(trg))

            if linkIdx in LL_CONN_POPULATIONS:
                connMap[src][LL_CONN_POPULATIONS.index(linkIdx)] += W_EXC_MAX

#     print("extracted connectivity map")
    ######################################################
    # 2. Convert spikes into cross-layer energy transfer
    ######################################################

    MIN_TIME = 0
    MAX_TIME = tstop
    times_discr = np.arange(MIN_TIME, MAX_TIME + dt, dt)
    micro_energy = np.zeros((len(LL_CONN_POPULATIONS), len(times_discr)))

    for iPop in range(N_POPULAITONS):
        # Only consider excitatory population data
        if (iPop % 2 == 0):
            tspk = nest.GetStatus(neuron_spike_detectors[iPop])[0]['events']['times']
            nspk = nest.GetStatus(neuron_spike_detectors[iPop])[0]['events']['senders']

            for t,n in zip(tspk, nspk):
                timeIdx = int((t - MIN_TIME)/dt)
                for iLink in range(len(LL_CONN_POPULATIONS)):
                    micro_energy[iLink][timeIdx] += connMap[n-1][iLink]


    for i in range(len(micro_energy)):
        nSourceNeuron = N_NEURON_PP[LL_CONN_POPULATIONS[i][0]]
        totEnergyTransfer = np.sum(micro_energy[i]) / (tstop / 1000)
        trueFunConn = totEnergyTransfer / nSourceNeuron
        outStr = "connection" + str(LL_CONN_REGIONS[i])
        outStr += " nSourceNeuron=" + str(nSourceNeuron)
        outStr += " trueFunConn=" + str(int(trueFunConn * 100)/100)
        logFile.write(outStr)


    plt.figure()
    for i in range(len(micro_energy)):
        plt.plot(times_discr, micro_energy[i])

    plt.savefig(outputName + '_EnergyT.png')
    plt.close()
    
    
    # Close everything
    logFile.close()

doing simulation 11
doing simulation 12
doing simulation 13
doing simulation 14
doing simulation 15
doing simulation 16
doing simulation 17
